# Set up flamegraph repo

In [ ]:
# Clone the FlameGraph repository from GitHub
!git clone https://github.com/brendangregg/FlameGraph.git

# Add the FlameGraph directory to the system's path
import os
flamegraph_dir = os.path.abspath('FlameGraph')
os.environ['PATH'] = f"{flamegraph_dir}:{os.environ['PATH']}"

# Verify that the FlameGraph scripts are available in the path
!which stackcollapse-perf.pl
!which flamegraph.pl

# Output a message indicating successful setup
print("FlameGraph setup complete!")


# CPU Analysis

In [ ]:
# Record CPU samples for a specified duration (e.g., 10 seconds)
!perf record -a -g -- sleep 10

# Generate a summary report of the CPU usage
!perf report --stdio

# Generate a flame graph to visualize the call stack and identify hot functions
!perf script | FlameGraph/stackcollapse-perf.pl | FlameGraph/flamegraph.pl > cpu_flamegraph.svg

# Display the flame graph in the notebook
from IPython.display import SVG
SVG(filename='cpu_flamegraph.svg')


# Off-CPU Analysis

In [ ]:
# Record off-CPU time samples for a specified duration (e.g., 10 seconds)
!perf record -e sched:sched_stat_sleep -a -g -- sleep 10

# Generate a flame graph to visualize off-CPU time call stacks
!perf script | stackcollapse-perf.pl | flamegraph.pl > offcpu_flamegraph.svg

# Display the flame graph in the notebook
SVG(filename='offcpu_flamegraph.svg')

# Memory Analysis

In [ ]:
# Record memory access samples for a specified duration (e.g., 10 seconds)
!perf record -e mem_inst_retired.all_loads -a -g -- sleep 10

# Generate a summary report of memory usage
!perf report --stdio

# Generate a flame graph to visualize memory access patterns
!perf script | FlameGraph/stackcollapse-perf.pl | FlameGraph/flamegraph.pl > memory_flamegraph.svg

# Display the flame graph in the notebook
SVG(filename='memory_flamegraph.svg')


# Memory Allocation Analysis

In [ ]:
# Record memory allocation samples for a specified duration (e.g., 10 seconds)
!perf record -e malloc -a -g -- sleep 10

# Generate a flame graph to visualize memory allocation call stacks
!perf script | stackcollapse-perf.pl | flamegraph.pl > memory_alloc_flamegraph.svg

# Display the flame graph in the notebook
SVG(filename='memory_alloc_flamegraph.svg')


# IO Analysis

In [ ]:
# Record block I/O events for a specified duration (e.g., 10 seconds)
!perf record -e block:block_rq_issue -a -g -- sleep 10

# Generate a summary report of I/O operations
!perf report --stdio

# Generate a flame graph to visualize I/O operations
!perf script | FlameGraph/stackcollapse-perf.pl | FlameGraph/flamegraph.pl > io_flamegraph.svg

# Display the flame graph in the notebook
SVG(filename='io_flamegraph.svg')


# Cache miss plot

In [ ]:
# Run this cell to collect L1 data cache miss data
!perf stat -e L1-dcache-load-misses -a -I 200 -- sleep 10 > cache_miss_data.txt
# Load and parse cache miss data from the file
def load_cache_miss_data(file_name):
    data = np.loadtxt(file_name, skiprows=1, usecols=1)
    return data

# Create cache miss plot
def create_cache_miss_plot(data):
    time = np.arange(0, len(data) * 0.2, 0.2)  # Assuming 200ms intervals
    plt.plot(time, data)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Cache Misses')
    plt.title('Cache Misses Over Time')
    plt.grid(True)
    plt.show()

# Load data and create cache miss plot
cache_miss_data = load_cache_miss_data('cache_miss_data.txt')
create_cache_miss_plot(cache_miss_data)


# Heatmap

In [ ]:
# Run this cell to collect CPU usage data for each CPU core
!perf stat -e cycles:u -a -A -I 200 -- sleep 10 > cpu_usage_data.txt

import matplotlib.pyplot as plt
import numpy as np

# Load and parse CPU usage data from the file
def load_cpu_usage_data(file_name):
    data = np.loadtxt(file_name, skiprows=1, usecols=range(1, 5))  # Adjust usecols based on the number of CPU cores
    return data

# Create heatmap for CPU usage
def create_heatmap(data):
    plt.imshow(data.T, cmap='hot', interpolation='nearest', aspect='auto')
    plt.colorbar(label='CPU Usage')
    plt.xlabel('Time')
    plt.ylabel('CPU Core')
    plt.title('CPU Usage Heatmap')
    plt.show()

# Load data and create heatmap
cpu_usage_data = load_cpu_usage_data('cpu_usage_data.txt')
create_heatmap(cpu_usage_data)
